In [ ]:
import clickhouse_connect

# Get client connection
client = clickhouse_connect.get_client(host='localhost', port=8123, username='default', password='default')


In [34]:
# Check existing tables in ClickHouse
print("Checking existing tables in ClickHouse...")
tables = client.query("SHOW TABLES").result_rows
print("\nCurrent tables:")
for table in tables:
    print(f"  - {table[0]}")


Checking existing tables in ClickHouse...

Current tables:
  - maid_geohash_state
  - mv_raw_to_geohash_agg
  - raw_maid_pings
  - view_aggregated_data


In [35]:
# Reset and initialize ClickHouse schema
print("Resetting ClickHouse schema...")

# Execute the ClickHouse schema script (includes DROP and CREATE statements)
with open('clickhouse_schema.sql', 'r') as f:
    schema_sql = f.read()

# Split by semicolons and execute each statement
statements = [stmt.strip() for stmt in schema_sql.split(';') if stmt.strip()]
for i, stmt in enumerate(statements, 1):
    try:
        client.command(stmt)
        # Only print progress for CREATE statements to reduce noise
        if 'CREATE' in stmt.upper():
            print(f"✅ Statement {i}/{len(statements)} executed successfully")
    except Exception as e:
        print(f"❌ Statement {i}/{len(statements)} failed: {str(e)}")
        print(f"Statement: {stmt[:100]}...")

print("\nSchema reset complete.")

# Show all tables in ClickHouse
tables = client.query("SHOW TABLES").result_rows
print("\nClickHouse Tables:")
for table in tables:
    print(f"  - {table[0]}")


Resetting ClickHouse schema...
✅ Statement 5/8 executed successfully
✅ Statement 6/8 executed successfully
✅ Statement 7/8 executed successfully
✅ Statement 8/8 executed successfully

Schema reset complete.

ClickHouse Tables:
  - maid_geohash_state
  - mv_raw_to_geohash_agg
  - raw_maid_pings
  - view_aggregated_data


In [36]:
import pandas as pd
# Clean tables
print("Cleaning tables...")
client.command("TRUNCATE TABLE IF EXISTS raw_maid_pings")

print("Importing multi_maids_sample.csv...")
df = pd.read_csv('multi_maids_sample.csv')
if 'geohash' in df.columns:
    df = df.drop(columns=['geohash'])
df['timestamp'] = df['timestamp'].astype(str).str.split('+').str[0]

# Insert data using client
try:
    client.insert_df('raw_maid_pings', df)
    print("✅ Insert Successful.")
    client.command("OPTIMIZE TABLE raw_maid_pings FINAL")
except Exception as e:
    print(f"❌ Insert Failed: {str(e)}")


Cleaning tables...
Importing multi_maids_sample.csv...
✅ Insert Successful.


In [37]:
result=client.query("SELECT * FROM view_aggregated_data").result_rows
import pandas as pd
res=pd.DataFrame(result)

In [38]:
res

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,u5rsznbSbEoot9Yele1xjGElkkAW6mi+7g4m4MuL80Eblw...,6,1410,"[f25d8xk, f256zcq, f256zcn, f25db9p, f25db47, ...","[640, 2, 661, 99, 2, 6]","[45.47993101477623, 45.49033546447754, 45.4900...","[-73.80005166530609, -73.82965087890625, -73.8...","[0.5750136797481376, -0.0, 0.6879205933512649,...","[0.435905854537778, 0.009343836882293574, 0.49...","[0.12880421620383953, 0.00950348737078084, 0.1...","[0.273938761893102, 0.05603712533303386, 0.479...","[0.6324287642397368, 0.0, 0.8202461299452188, ...","[13.391543297603697, 4.763676447913455, 5.0908..."
1,ONlX8Z6S5WTYGkAqIYJiXXBUF1AuZNj2giQAbzJG1v4kjn...,12,1877,"[eyh3tyz, ev0fhkq, ey51kqf, ey7gs0z, ey50fzy, ...","[16, 390, 452, 53, 96, 4, 15, 3, 28, 809, 3, 8]","[34.05080032348633, 28.499999041435046, 34.007...","[-5.009921997785568, -9.99999966499133, -6.840...","[0.5416485481667066, 0.7126630151479483, 0.739...","[0.012619439522523132, 0.25642838414740304, 0....","[0.039561573220380036, 0.3614007766394031, 0.2...","[0.17382034690664155, 0.20356089619104586, 0.3...","[0.6810808170252423, 0.8441610840236423, 0.798...","[2.950201933562846, 3.6536917477251714, 5.4505..."
2,HEHyPZs7dFuBcXKOwwOhyBLf6QY/CvxYFqleS/MfxpXeVA...,23,7297,"[evg6fqj, evg6fqv, evg6fqy, evg6fqe, evg6fmv, ...","[133, 10, 96, 226, 110, 71, 76, 2357, 1072, 68...","[32.86096948609316, 32.86554107666016, 32.8653...","[-6.573182923453195, -6.572651386260986, -6.57...","[0.050414351222475076, -0.0, -0.0, 0.197954963...","[0.043003671290368015, 0.021785782663859863, 0...","[0.14753248707485112, 0.1564609729827798, 0.18...","[0.29440265480800126, 0.10802929162330824, 0.1...","[0.29238954977944437, 0.5469548096692048, 0.31...","[53.521405202560096, 6.66438407125583, 46.0010..."
3,ePnYqFRpG/TwW8b1J50d5MpucIJa839aNeeF7calsX9pk8...,219,2338,"[evdk1ue, evdk1fm, evdk4p0, evdk604, evdk4j0, ...","[3, 2, 4, 1, 1, 6, 1, 2, 1, 9, 1, 8, 4, 16, 2,...","[31.6662114461263, 31.6533842086792, 31.680192...","[-8.004474957784018, -8.002094268798828, -7.99...","[-0.0, 0.21810250817884405, 0.2181025081788440...","[0.02304882885955386, 0.05921481586970782, 0.0...","[0.022909786521373925, 0.05119724621616454, 0....","[0.13283655832307614, 0.04235078704257574, 0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.06509947454480276,...","[38.538337478335514, 18.43898073814728, 23.602..."
4,KBO16cQcRhZ0D30YbAcG0+1kUUCafDDCNhhAe+a5YPsAgY...,183,1602,"[evfwfkc, evfwd7s, evfw9cd, evfwdmc, evfw93w, ...","[7, 1, 2, 1, 1, 2, 1, 1, 2, 1, 5, 1, 1, 1, 14,...","[33.55646569388254, 33.50666427612305, 33.4950...","[-7.63332428250994, -7.629642963409424, -7.654...","[0.549976147512174, -0.0, 0.21810250817884405,...","[0.06701996971751398, 0.005208640598821776, 0....","[0.10453465541191713, 0.005397049170301682, 0....","[0.16642237401517931, 0.02663459077343874, 0.0...","[0.3801496501199269, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[13.634657983581956, 0.0, 36.974942849448816, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,vPgTSidpH487at1YiMiSjTJyAxbhlHpn4EcBRAoDnJ/6Zt...,482,1536,"[dpzcv7c, drbffu3, 9g3qpxb, sggf784, suk5jkg, ...","[1, 4, 1, 5, 2, 3, 1, 2, 1, 6, 2, 2, 1, 2, 1, ...","[43.92279815673828, 44.1011323928833, 19.37878...","[-78.86930084228516, -77.57292938232422, -99.1...","[-0.0, 0.21810250817884405, -0.0, 0.5064183413...","[0.026190504240902256, 0.09572204114773249, 0....","[0.005397049170301682, 0.09319887263115623, 0....","[0.026472870470701855, 0.1135862997189375, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0559219776122524, 0.0, 1.3568221471190..."
96,udqhgo/+e7Kv3d7SZNU+z5aR7tVS/N+fsb/PimP9PqY/LC...,1403,2507,"[ey537jg, eyh3mwz, eyh06qs, ey5bxu9, ey5by1s, ...","[1, 3, 1, 1, 1, 3, 5, 2, 3, 2, 1, 1, 1, 1, 1, ...","[34.00193786621094, 34.00746281941732, 33.8304...","[-6.542697429656982, -5.021676858266194, -5.52...","[-0.0, -0.0, -0.0, -0.0, -0.0, 0.2002826246574...","[0.005208640598821776, 0.012361746202427276, 0...","[0.02980452320292592, 0.07836126094921189, 0.0...","[0.026446467155969305, 0.061180